# Alternativas ULSP

Veamos ahora un par de implementaciones de algoritmos para resolver el ULSP que no requieren de programación entera.

También veremos un poco la evaluación empírica de la eficiencia de un algoritmo

In [ ]:
using Random,Distributions
Random.seed!(100003) ### introducir una semilla de números pseudo-aleatorios

In [ ]:
#instancia del ulsp -> número de periodos, un coste K y un coste h
#distribución normal
function generarInstanciaNormal(periodos,ratioFijoVariable,mu,sigma)
    precioFijo=ratioFijoVariable
    precioStock=1
    d=mu.+randn(periodos).*sigma #utilizando notación vectorial he calculado la demanda
    #randn(periodos) -> N(0,1) --> N(mu,\sigma^2) mu+z*sigma los puntos son para hacerlo con vector
    demanda=convert.(Int,round.(d,digits=0)) #prefiero trabajar con valores enteros
    for i in 1:periodos
        if demanda[i]<1
            demanda[i]=1
        end
    end
    #se puede sustituir demanda=convert.(Int,round.(d,digits=0)) por:
    #demanda=zeros(Int,periodos)
    #for i in 1:periodos
    #    valor=round(d[i],digits=0)
    #    demanda[i]=convert(Int,d[i])
    #end
    return precioFijo,precioStock,demanda
end
#distribución uniforme
function generarInstanciaUniforme(periodos,ratioFijoVariable,minimo,maximo)
    precioFijo=ratioFijoVariable
    precioStock=1
    d=rand(DiscreteUniform(minimo,maximo), periodos) #utilizando notación vectorial he calculado la demanda
    #randn(periodos) -> N(0,1) --> N(mu,\sigma^2) mu+z*sigma los puntos son para hacerlo con vector
    demanda=convert.(Int,round.(d,digits=0)) #prefiero trabajar con valores enteros
    #se puede sustituir demanda=convert.(Int,round.(d,digits=0)) por:
    #demanda=zeros(Int,periodos)
    #for i in 1:periodos
    #    valor=round(d[i],digits=0)
    #    demanda[i]=convert(Int,d[i])
    #end
    return precioFijo,precioStock,demanda
end

In [ ]:
#Wagner-Whitin
function ww(K,h,d,T)
    c=zeros(Int,T,T+1) #matriz de costes de fila a columna
    for i in 1:T
        for j in i+1:T+1
            compra=sum(d[i:j-1])
            c[i,j]=K
            for k in i:j-1
                compra -= d[k]
                c[i,j]=c[i,j]+h*compra
            end
        end
    end
    f=zeros(Int,T+1) #recordar que el periodo T+1 es el periodo ω
    ω=T+1
    f[ω]=0
    for t in T:-1:1
        # $f_t= \min_{k>t} (c_{tk}+f_k) $
        f[t]=typemax(Int32)
        for k in t+1:T+1
            if f[t]>(c[t,k]+f[k])
                f[t]=c[t,k]+f[k]
            end
        end
    end
    return(f[1])
end
#test
ww(K,h,d,nPeriodos)

In [ ]:
#Shortest Paths
function sp(K,h,d,T)
    c=zeros(Int,T,T+1) #matriz de costes de fila a columna
    for i in 1:T
        for j in i+1:T+1
            compra=sum(d[i:j-1])
            c[i,j]=K
            for k in i:j-1
                compra -= d[k]
                c[i,j]=c[i,j]+h*compra
            end
        end
    end
    f=zeros(Int,T+1) #recordar que el periodo T+1 es el periodo ω
    f[1]=0
    for t in 2:T+1
        # $f_t= \min_{k<t} (c_{kt}+f_k) $
        f[t]=typemax(Int32)
        for k in 1:t-1
            if f[t]>(f[k]+c[k,t])
                f[t]=f[k]+c[k,t]
            end
        end
    end
    return(f[T+1])
end
sp(K,h,d,nPeriodos)

In [ ]:
#Silver-Meal
function sm(K,h,d,T)
    accCoste=0
    lastCompra=1
    costeAnterior=K
    costeActual=K
    accCoste=K
    for t in 2:T
        costeActual += (t-lastCompra)*h*d[t]
        if costeActual/(t+1-lastCompra) <= costeAnterior
            costeAnterior=costeActual/(t+1-lastCompra)
            accCoste += (t-lastCompra)*h*d[t]
        else
            accCoste += K
            costeAnterior=K
            costeActual=K
            lastCompra=t
        end
    end
    return accCoste
end
sm(K,h,d,nPeriodos)

In [ ]:
maxTrials=10
for nPeriodos in 5:250
    sumGaps=0.0
    for trials in 1:maxTrials
        K,h,d=generarInstanciaNormal(nPeriodos,200,100,75)
        Vww=ww(K,h,d,nPeriodos)
        Vsp=sp(K,h,d,nPeriodos)
        Vsm=sm(K,h,d,nPeriodos)
        @assert(Vww==Vsp)
        @assert(Vww<=Vsm)
        gap=(Vsm-Vsp)/(Vsp)
        sumGaps += gap
    end
    println(nPeriodos,"\t",round(sumGaps/100.0*maxTrials; digits=2))
end

In [ ]:
maxTrials=10
for nPeriodos in 5:250
    sumGaps=zeros(Float64,5)
    for rangoCostes in 1:5
        for trials in 1:maxTrials
            K,h,d=generarInstanciaNormal(nPeriodos,100*rangoCostes,100,75)
            Vww=ww(K,h,d,nPeriodos)
            Vsp=sp(K,h,d,nPeriodos)
            Vsm=sm(K,h,d,nPeriodos)
            @assert(Vww==Vsp)
            @assert(Vww<=Vsm)
            gap=(Vsm-Vsp)/(Vsp)
            sumGaps[rangoCostes] += gap
        end
    end
    print(nPeriodos)
    for i in 1:5
        print("\t",round(sumGaps[i]/100.0*maxTrials; digits=2))
    end
    println()
end